# Prepairing chesscom data to use

## Import and functions

In [1]:
# coding: utf-8

# our all
import numpy as np
import pandas as pd

# usefull pandas settings
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# for API working and current time
import requests
import datetime

# chess pgn-reading tool
from pgn_parser import pgn, parser

# multistreaming
import threading

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

In [2]:
# for logging our process
def logging(proc, par):
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    import json
    params_dict={}
    for val in par:
        params_dict.update(val)
        
    params = json.dumps(params_dict)

    log_df=pd.DataFrame([now, proc, params]).T
    log_df.columns=['time', 'proc', 'params']

    log_df.to_csv('log.csv', index=False, header=False, mode='a+')

In [3]:
# convert unixtime to time:
# fr_unixtime(1565211491) -> '2019-08-07 20:58:11'
def fr_unixtime(ts):
    from datetime import datetime
    return datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d %H:%M:%S')

# convert integer month number to string API format:
# 1 -> '01' 
# 9 -> '09' 
# 12 > '12'
# 112 -> -1
def get_number(x):
    if x>=10 and x<100:
        return str(x)
    elif x<10:
        return '0'+str(x)
    else:
        return -1
    
# retunr classoc Elo propabilities
# elo_prob(2882, 2722) -> 0.7152 (72% chanses Carlsen (2882) to beat Wan Hao (2722))
def elo_prob(rw, rb):
    return 1/(1+np.power(10, (rb-rw)/400))


# working with pgn
# pip install pgn-parser
def get_pgn(text):
    from pgn_parser import pgn, parser
    game = parser.parse(text, actions=pgn.Actions())
    
    try:
        score=game.tag_pairs['Result']
    except:
        score='Unknown'  
    try:
        date=game.tag_pairs['Date']
    except:
        date='Unknown'
    try:
        time=game.tag_pairs['UTCTime']
    except:
        time='Unknown'
    try:
        eco=game.tag_pairs['ECO']
    except:
        eco='Unknown'
    try:
        ecourl=game.tag_pairs['ECOUrl']
    except:
        ecourl='Unknown'
    return {'score':score, 
            'date': date, 
            'time':time, 
            'ECO':eco, 
            'ECO_url':ecourl}

# functions for predictions
# find col with target user (white or black)
def find_col(target_user, col1, wh_val, bl_val):
    if col1==target_user:
        return wh_val
    else:
        return bl_val
# reverse fun of find_col    
def find_opp(target_user, col1, wh_val, bl_val):
    if col1!=target_user:
        return wh_val
    else:
        return bl_val

In [4]:
# get json form site and return string
# exmple get_api_data_to_str('eric', '2014', '01')
def get_api_data_to_str(player, year, month):
    import requests
    response = requests.get('https://api.chess.com/pub/player/'+player+'/games/'+year+'/'+month)
    st=response.text
    
    if response.status_code ==200:
        proc='api_call_sucess'
        logging(proc, 
                [
                    {'player':player},
                    {'year': year},
                    {'month': month},
                    {'type': 'norm'}
                ]
               )
    if response.status_code !=200:
        proc='api_call_error'
        logging(proc, 
                [
                    {'player':player},
                    {'year': year},
                    {'month': month},
                    {'type': 'error'},
                    {'error_code': response.status_code},
                    {'error_message': st},
                ]
               )
    return st, response.status_code

# convert api data to pandas for next working
def get_str_data_to_pandas(strng):
    import json
    try:
        js = json.loads(strng)
        df=pd.DataFrame(js)
    except:
        proc='data_parse_error'
        logging(proc, 
                [
                    {'type': 'error'},
                    {'source': strng[0:3000]}
                ]
               )       
    return df

# convert multilevel json to table
def prepair_pandas_multilevel_data(df):
    num=len(df)
    new_df=pd.DataFrame()
    for i in range(num):
        b=pd.io.json.json_normalize(df.values[i])
        new_df=pd.concat([new_df, b])
    return new_df 

# get player games in month with preparation
def get_pl_stat(game_stat):    
    # chess result dictionary
    res_dict={
    'win': 1.0,
    'checkmated': 0.0,
    'agreed': 0.5,
    'repetition': 0.5,
    'timeout': 0.0,
    'resigned': 0.0,
    'stalemate': 0.5,
    'lose': 0.0,
    'insufficient': 0.5,
    '50move': 0.5,
    'abandoned': 0.0,
    'kingofthehill': 0.5,
    'threecheck': 0.5,
    'timevsinsufficient': 0.5,
    'bughousepartnerlose': 0.0
    }

    # number of games
    num=len(game_stat)
    
    # cut long and unusefull columns
    game_stat=game_stat[['rules', 'time_class', 'time_control', 'rated',
        'white.@id', 'white.rating', 'white.result', 'white.username',
         'black.@id', 'black.rating', 'black.result', 'black.username',
         'end_time', 'pgn', 
         'url']]
    # score 0.0, 0.5 or 1.0
    game_stat['white.score']=game_stat['white.result'].map(res_dict)
    game_stat['black.score']=game_stat['black.result'].map(res_dict)
    
    # pgn - long string
    game_stat['pgn']=game_stat['pgn'].apply(get_pgn)
    # result - string like '1-0'
    game_stat['result']=game_stat['pgn'].apply(lambda x: x['score'])
    
    game_stat['date']=game_stat['pgn'].apply(lambda x: x['date'])
    game_stat['time']=game_stat['pgn'].apply(lambda x: x['time'])
    
    # ECO - string like 'B10'
    game_stat['eco']=game_stat['pgn'].apply(lambda x: x['ECO'])
    # ECOurl - url like 'https://www.chess.com/openings/B10-Caro-Kann-Defense-2.Nf3-d5'
    game_stat['eco_url']=game_stat['pgn'].apply(lambda x: x['ECO_url'])
    
    # propability from classic formula
    game_stat['white_elo_forecast']=game_stat[['white.rating', 'black.rating']].apply(lambda x: elo_prob(*x), axis=1)
    game_stat['black_elo_forecast']=game_stat[['black.rating', 'white.rating']].apply(lambda x: elo_prob(*x), axis=1)
    
    # after it np.sum(df['game']) means number of games
    game_stat['game']=1
    game_stat['date'] = game_stat['date'].astype('datetime64[ns]')
    # period - stirng like '2019-08'
    game_stat['period']=game_stat['date'].dt.year.astype('str')+'-'+game_stat['date'].dt.month.astype('str')
        
    game_stat.columns=['rules', 'time_class', 'time_control', 'rated', 'white_url',
       'white.rating', 'white.result', 'white.username', 'black_url',
       'black.rating', 'black.result', 'black.username', 'end_time', 'pgn', 'game_url',
       'white.score', 'black.score', 'result', 'date', 'time', 'eco',
       'eco_url', 'white_elo_forecast', 'black_elo_forecast',
       'game', 'period']
    game_stat=game_stat[[
        'rules', 'time_class', 'time_control', 'rated', 'game',
        'result', 'date', 'time', 'period',
        'white.username', 'black.username',
        'white.rating', 'black.rating', 
        'white.result', 'black.result', 
        'white.score', 'black.score', 
        'white_elo_forecast', 'black_elo_forecast',
        'eco', 'eco_url', 
        'game_url', 'white_url', 'black_url'
    ]]
    
    
    game_stat['date'] = game_stat['date'].astype('datetime64[ns]')
    proc='pl_stat'
    player=game_stat['white.username'].value_counts().index[0]
    year=game_stat['date'].value_counts().index[0].strftime("%Y")
    month=game_stat['date'].value_counts().index[0].strftime("%m")
    logging(proc, 
        [
            {'player':player},
            {'year': year},
            {'month': month},
            {'type': 'norm'}
        ]
       )
    return game_stat
    


In [89]:
def status(player, year, month, status, is_error):
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    buf_df=pd.DataFrame([[player, year, month, now, status, is_error]], 
                        columns=['player', 'year', 'month', 'time','status', 'is_error'])
    buf_df.to_csv('data/status.csv', index=False, header=False, mode='a+')    

In [143]:
# main function
def get_btch(player, year, month):
    import datetime
    import time
    import json

    status(player, year, month, 'new', is_error=0)
    
    
    # list random seconds of delay
    delay_lst=[np.round(2*np.random.rand(1,1)[0,0],0), 
               np.round(4*np.random.rand(1,1)[0,0],0), 
               np.round(10*np.random.rand(1,1)[0,0],0), ]
    
    # we give 3 attemps to get data
    for cs in range(3):    
        time.sleep(delay_lst[cs])
        # working with propably exception
        try:
            # API -> string
            data_string, code=get_api_data_to_str(player, year, month)
        except:
            data_string=''
            code=500
        # after it we need to check status code
        # case best variant
        if code==200:
            proc='api_call_saved'
            logging(proc, 
                    [
                        {'player':player},
                        {'year': year},
                        {'month': month},
                        {'type': 'norm'}
                    ]
                   )
            # exit from cicle 
            break
        # else log and try again
        else:
            api_error_str='api error, code: '+str(code)+', message: '+data_string[0:3000]+'case: '+str(cs)
            status(player, year, month, api_error_str, is_error=1)
            proc='api_call_error_recieved'
            logging(proc, 
                    [
                        {'player':player},
                        {'year': year},
                        {'month': month},
                        {'type': 'error'},
                        {'try':cs},
                        {'error_code': code},
                        {'error_message': data_string[0:3000]}
                    ]
                   ) 
        # at the end we need to log the end of our attepts
        if cs==2:
            proc='api_call_final_error'
            logging(proc, 
                    [
                        {'player':player},
                        {'year': year},
                        {'month': month},
                        {'type': 'error'},
                        {'try':cs},
                        {'error_code': code},
                        {'error_message': data_string[0:3000]}
                    ]
                   )
    
    # string -> json -> DataFrame
    try:
        multidata=get_str_data_to_pandas(data_string)
    except:
        status(player, year, month, 'error_to_pandas', is_error=1)
    
    # DataFrame -> use json normalise -> good df
    try:
        usefull_data=prepair_pandas_multilevel_data(multidata)
    except:
        status(player, year, month, 'multilevel error', is_error=1)
    
    # game prepair
    try:
        workng_data=get_pl_stat(usefull_data)
    except:
        status(player, year, month, 'coverting error', is_error=1)
    
    # save to file
    try:
        now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        workng_data.to_csv('source/'+player+'_'+year+'_'+month+'_'+now+'.csv')
        suc_str='successfull get data: '+str(len(workng_data))+' rows'
        status(player, year, month, suc_str, is_error=0)
    except:
        status(player, year, month, 'writing error', is_error=1)
        
    return multidata

In [114]:
get_btch('rosolimo','2019', '10')

In [157]:
df=pd.read_csv('data/status.csv', header=None)
df.columns=['player', 'year', 'month', 'time','status', 'is_error']

In [158]:
df.sample(3)

,player,year,month,time,status,is_error
3916,00zen00,2019,1,2019-09-01 18:34:16,writing error,1
4459,019325,2019,10,2019-09-01 18:34:24,writing error,1
2986,001r78,2019,12,2019-09-01 18:34:04,multilevel error,1


In [160]:
len(df)

5241

In [159]:
df.groupby(['player', 'year', 'month']).max()[['status', 'time', 'is_error']]

status  \
player         year month                                    
0000000000yyyy 2018 1                        writing error   
                    2                        writing error   
                    3                        writing error   
                    4                        writing error   
                    5                        writing error   
                    6                        writing error   
                    7                        writing error   
                    8                        writing error   
                    9                        writing error   
                    10                       writing error   
                    11                       writing error   
                    12                       writing error   
               2019 1                        writing error   
                    2                        writing error   
                    3                        writing error   
                    4                        writing error   
                    5                        writing error   
                    6                        writing error   
                    7                        writing error   
                    8        successfull get data: 51 rows   
                    9         successfull get data: 1 rows   
                    10                       writing error   
                    11                       writing error   
                    12                       writing error   
00001111romeo  2018 1                        writing error   
                    2                        writing error   
                    3                        writing error   
                    4                        writing error   
                    5                        writing error   
                    6                        writing error   
                    7                        writing error   
                    8                        writing error   
                    9                        writing error   
                    10                       writing error   
                    11                       writing error   
                    12                       writing error   
               2019 1                        writing error   
                    2                        writing error   
                    3                        writing error   
                    4                        writing error   
                    5                        writing error   
                    6                        writing error   
                    7                        writing error   
                    8                        writing error   
                    9         successfull get data: 1 rows   
                    10                       writing error   
                    11                       writing error   
                    12                       writing error   
000evgen       2018 1                        writing error   
                    2                        writing error   
                    3                        writing error   
                    4                        writing error   
                    5       successfull get data: 168 rows   
                    6        successfull get data: 30 rows   
                    7                        writing error   
                    8                        writing error   
                    9                        writing error   
                    10                       writing error   
                    11                       writing error   
                    12        successfull get data: 1 rows   
               2019 1                        writing error   
                    2                        writing error   
                    3        successfull get data: 88 rows   
                    4 

In [161]:
df[df['is_error']==1].groupby(['player', 'year', 'month']).min()[['status', 'time']]

status  \
player         year month                                                                                                  
0000000000yyyy 2018 1                                                             api error, code: 429, message: case: 0   
                    2                                                             api error, code: 429, message: case: 0   
                    3                                                                                    coverting error   
                    4                                                             api error, code: 429, message: case: 0   
                    5                                                                                    coverting error   
                    6                                                             api error, code: 429, message: case: 0   
                    7                                                                                    coverting error   
                    8                                                             api error, code: 429, message: case: 0   
                    9                                                             api error, code: 429, message: case: 0   
                    10                                                            api error, code: 429, message: case: 0   
                    11                                                            api error, code: 429, message: case: 0   
                    12                                                            api error, code: 429, message: case: 0   
               2019 1                                                             api error, code: 429, message: case: 0   
                    2                                                             api error, code: 429, message: case: 0   
                    3                                                             api error, code: 429, message: case: 0   
                    4                                                             api error, code: 429, message: case: 0   
                    5                                                                                    coverting error   
                    6                                                                                    coverting error   
                    7                                                             api error, code: 429, message: case: 0   
                    10                                                            api error, code: 429, message: case: 0   
                    11                                                            api error, code: 429, message: case: 0   
                    12                                                            api error, code: 429, message: case: 0   
00001111romeo  2018 1                                                             api error, code: 429, message: case: 0   
                    2                                                             api error, code: 429, message: case: 0   
                    3                                                             api error, code: 429, message: case: 0   
                    4                                                                                    coverting error   
                    5                                                             api error, code: 429, message: case: 0   
                    6                                                                                    coverting error   
                    7                                                             api error, code: 429, message: case: 0   
                    8                                                                                    coverting error   
                    9                                                             api error, code: 429, message: case: 0   
                    10

In [162]:
len(df[df['is_error']==1].groupby(['player', 'year', 'month']).min()[['status', 'time']])

760

In [167]:
df[(df['year']==2018)&(df['month']==10)&(df['player']=='02klemento02')]

,player,year,month,time,status,is_error
759,02klemento02,2018,10,2019-09-01 18:33:19,new,0
1541,02klemento02,2018,10,2019-09-01 18:33:29,"api error, code: 429, message: case: 0",1
1892,02klemento02,2018,10,2019-09-01 18:33:41,"api error, code: 429, message: case: 1",1
2659,02klemento02,2018,10,2019-09-01 18:33:57,"api error, code: 429, message: case: 2",1
3288,02klemento02,2018,10,2019-09-01 18:34:08,error_to_pandas,1
3445,02klemento02,2018,10,2019-09-01 18:34:11,multilevel error,1
3687,02klemento02,2018,10,2019-09-01 18:34:13,coverting error,1
4458,02klemento02,2018,10,2019-09-01 18:34:16,writing error,1


In [164]:
# start getting with threading magic
def user_parse(user_list, year_range):   
    import threading
    import json
    
    i=0
    for user in user_list:
        for year in year_range:
            for month in range(1,13):
                t = threading.Thread(target=get_btch, args=(user, str(year), get_number(month)))
                t.start()
                

                i=i+1
    print('All btch', i,  'are started')   


In [133]:
year_range=range(2012, 2020)

In [134]:
user_parse(['rosolimo'], year_range)

All btch 96 are started


In [9]:
# read all json in folder and concate DataFrame
def read_files(path):
    # reading
    l=[]
    cntr=0
    import os
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".csv"):
                 l.append(os.path.join(root, file))
            cntr=cntr+1
    print('Total', cntr, 'files founded')

    # merging
    i=0
    df=pd.DataFrame()
    for link in l:
        try:
            dfb=pd.read_csv(link)
        except:
            dfb=pd.DataFrame()
        df=pd.concat([df, dfb])
        
    df=df.drop_duplicates()
    return df

In [10]:
# data for predict current user
def learn_prepair(df, target_user):
    df=df[(df['white.username']==target_user)|(df['black.username']==target_user)]
    df['target_user']=target_user
    df['score']=df[['target_user', 'white.username', 'white.score', 'black.score']].apply(lambda x: find_col(*x), axis=1)
    df['rating']=df[['target_user','white.username', 'white.rating', 'black.rating']].apply(lambda x: find_col(*x), axis=1)
    df['opponent_rating']=df[['target_user', 'white.username', 'white.rating', 'black.rating']].apply(lambda x: find_opp(*x), axis=1)
    df['base_elo_forec']=df[['target_user', 'white.username', 'white_elo_forecast', 'black_elo_forecast']].apply(lambda x: find_col(*x), axis=1)
    df['color']=np.where(df['white.username']==target_user, 'w', 'b')
    
    df['date'] = df['date'].astype('datetime64[ns]')
    df['year']=df['date'].dt.year
    df['month']=df['date'].dt.month
    
    return df

In [11]:
# get players list by countries
def get_county_players_list(country):
    import requests
    response = requests.get('https://api.chess.com/pub/country/'+country+'/players')
    st=response.text
    if response.status_code !=200:
        print(response.status_code)
        print(st[0:3000])
        
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
 
    try:
        file = open('player_lists/players_list_'+country+'_'+now+'.json','w') 
        file.write(st)
        file.close()
    except Exception:
        error_st=response.status_code+st[0:3000]
        file = open('player_lists/players_list_'+country+'_'+now+'.json','w') 
        file.write( error_st)
        file.close()


## Start working

In [152]:
# collect players by countries
# countries_list=['RU']
# get_county_players_list(countries_list[0])

user_list=pd.read_json('player_lists/players_list_RU_2019-08-25 10:47:56.json')['players'].values
# user_list=['andreyvict', 'sever043', 'Rosolimo']

In [155]:
user_list[0:30]

array(['0000000000yyyy', '00001111romeo', '000evgen', '00130712345',
       '001mexak', '001r78', '003322', '003arthur300', '007-bont',
       '007go', '007vasili7', '009177', '0093374947', '00count00',
       '00mrx00', '00zen00', '01246', '0179323', '019283745', '019325',
       '01burunduk02', '01roman', '01shkv01', '02021971', '02081928',
       '02102013', '02110', '02klemento02', '02_qeka_02', '030362'],
      dtype=object)

In [154]:
len(user_list)

36950

In [156]:
year_range=range(2018, 2020)
# user_parse(user_list[4:5], year_range)
user_parse(user_list[0:30], year_range)

All btch 720 are started


Exception in thread Thread-190:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-427:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-701:
Traceback (most recent call last):
  File "/home/roman/anaco

Exception in thread Thread-448:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-273:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-604:
Traceback (most recent call last):
  File "/home/roman/anaco

Exception in thread Thread-235:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-245:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-328:
Traceback (most recent call last):
  File "/home/roman/anaco

Exception in thread Thread-420:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-293:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-258:
Traceback (most recent call last):
  File "/home/roman/anaco

...

Exception in thread Thread-837:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-739:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-532:
Traceback (most recent call last):
  File "/home/roman/anaco

Exception in thread Thread-327:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-400:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-760:
Traceback (most recent call last):
  File "/home/roman/anaco

Exception in thread Thread-559:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-697:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-774:
Traceback (most recent call last):
  File "/home/roman/anaco

Exception in thread Thread-868:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-671:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-766:
Traceback (most recent call last):
  File "/home/roman/anaco

Exception in thread Thread-603:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-699:
Traceback (most recent call last):
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-143-946210450f5d>", line 97, in get_btch
    return multidata
UnboundLocalError: local variable 'multidata' referenced before assignment

Exception in thread Thread-438:
Traceback (most recent call last):
  File "/home/roman/anaco

In [20]:
get_api_data_to_str('000evgen', '2018', '01')

('{"games":[]}', 200)

In [47]:
%%time
# df=read_files('source/')
df=pd.read_csv('source_old/10net_2018_08_2019-08-11 20:38:18.csv')
df['date'] = df['date'].astype('datetime64[ns]')

CPU times: user 10.9 ms, sys: 0 ns, total: 10.9 ms
Wall time: 12.2 ms


In [48]:
len(df)

95

In [49]:
df.sample(6)

,Unnamed: 0,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url
23,0,chess,blitz,180+2,True,1,0-1,2018-08-05,13:59:01,2018-8,BenMuhit,10net,2052,1988,resigned,win,0.0,1.0,0.591076,0.408924,D30,https://www.chess.com/openings/D30-Queens-Gambit-Declined,https://www.chess.com/live/game/2985840136,https://api.chess.com/pub/player/benmuhit,https://api.chess.com/pub/player/10net
76,0,chess,blitz,180+2,True,1,1-0,2018-08-17,18:57:27,2018-8,10net,aleks80,2080,2109,win,timeout,1.0,0.0,0.458362,0.541638,E76,https://www.chess.com/openings/E76-Kings-Indian-Defense-Four-Pawns-Dynamic-Line,https://www.chess.com/live/game/3012925085,https://api.chess.com/pub/player/10net,https://api.chess.com/pub/player/aleks80
18,0,chess,blitz,180+2,True,1,0-1,2018-08-04,14:58:48,2018-8,pyrta,10net,1903,1928,resigned,win,0.0,1.0,0.464084,0.535916,A40,https://www.chess.com/openings/A40-Queens-Pawn-Opening-Horwitz-Defense-2.Bf4,https://www.chess.com/live/game/2983830706,https://api.chess.com/pub/player/pyrta,https://api.chess.com/pub/player/10net
79,0,chess,blitz,180+2,True,1,1-0,2018-08-17,21:24:13,2018-8,10net,CandidateMaster20,2096,1955,win,checkmated,1.0,0.0,0.692463,0.307537,D85,https://www.chess.com/openings/D85-Gruenfeld-Defense-Exchange-Kemeri-Variation,https://www.chess.com/live/game/3013198698,https://api.chess.com/pub/player/10net,https://api.chess.com/pub/player/candidatemaster20
67,0,chess,blitz,180+2,True,1,0-1,2018-08-17,00:44:06,2018-8,mapanganibII,10net,2029,2079,timeout,win,0.0,1.0,0.428537,0.571463,D30,https://www.chess.com/openings/D30-Queens-Gambit-Declined,https://www.chess.com/live/game/3011253288,https://api.chess.com/pub/player/mapanganibii,https://api.chess.com/pub/player/10net
84,0,chess,blitz,180+2,True,1,1/2-1/2,2018-08-20,20:44:32,2018-8,10net,PawnEatingZombie,2117,2138,agreed,agreed,0.5,0.5,0.469815,0.530185,D31,https://www.chess.com/openings/D31-Queens-Gambit-Declined-Queens-Knight-Variation-3...Bb4,https://www.chess.com/live/game/3019747124,https://api.chess.com/pub/player/10net,https://api.chess.com/pub/player/pawneatingzombie


In [50]:
df['white.username'].value_counts().index[0]

'10net'

In [73]:
df['date'].value_counts().index[0].strftime("%m")

'08'

In [65]:
a

Timestamp('2018-08-17 00:00:00')

In [69]:
from datetime import datetime
tstr=a.strftime("%Y")

In [70]:
tstr

'2018'

In [52]:
df['date'].value_counts().index[0][5:7]

TypeError: 'Timestamp' object is not subscriptable

In [19]:
log_df=pd.read_csv('log.csv', header=None)
log_df.columns=['time', 'proc', 'params']
log_df['line']=1

In [20]:
len(log_df)

132

In [21]:
log_df.sample(3)

,time,proc,params,line
83,2019-08-25 01:27:24,api_call_error,"{""player"": ""Rosolimo"", ""year"": ""2019"", ""month"": ""12"", ""error_code"": 404, ""error_message"": ""{\""message\"":\""Date cannot be set in the future\"",\""code\"":0}""}",1
98,2019-08-25 01:27:26,api_call_sucess,"{""player"": ""Rosolimo"", ""year"": ""2019"", ""month"": ""05""}",1
20,2019-08-25 01:27:23,api_call_error,"{""player"": ""sever043"", ""year"": ""2019"", ""month"": ""12"", ""error_code"": 404, ""error_message"": ""{\""message\"":\""Date cannot be set in the future\"",\""code\"":0}""}",1


In [22]:
log_df.groupby('proc').sum()

,line
proc,
api_call_error,36
api_call_sucess,72
btch_finished,24


In [111]:
import json
import ast
log_suc=log_df[log_df['proc']=='api_call_sucess'].reset_index()
log_suc['params']=log_suc['params'].apply(lambda x: ast.literal_eval(x))

params=pd.io.json.json_normalize(log_suc['params'])
log_suc=pd.concat([log_suc, params], axis=1)
log_suc=log_suc[pd.isnull(log_suc['time'])==0]
log_suc=log_suc.drop(columns='params')

log_err=log_df[log_df['proc']=='api_call_error'].reset_index()
log_err['params']=log_err['params'].apply(lambda x: ast.literal_eval(x))

params=pd.io.json.json_normalize(log_err['params'])
log_err=pd.concat([log_err, params], axis=1)
log_err=log_err[pd.isnull(log_err['time'])==0]
log_err=log_err.drop(columns='params')

In [113]:
len(log_err)

36

In [114]:
log_err

,index,time,proc,line,error_code,error_message,month,player,year
0,2,2019-08-25 00:40:48,api_call_error,1,404,"{""message"":""Date cannot be set in the future"",""code"":0}",09,andreyvict,2019
1,8,2019-08-25 00:40:48,api_call_error,1,404,"{""message"":""Date cannot be set in the future"",""code"":0}",11,andreyvict,2019
2,9,2019-08-25 00:40:48,api_call_error,1,404,"{""message"":""Date cannot be set in the future"",""code"":0}",10,sever043,2019
3,13,2019-08-25 00:40:48,api_call_error,1,404,"{""message"":""Date cannot be set in the future"",""code"":0}",09,sever043,2019
4,16,2019-08-25 00:40:48,api_call_error,1,404,"{""message"":""Date cannot be set in the future"",""code"":0}",10,andreyvict,2019
5,18,2019-08-25 00:40:48,api_call_error,1,404,"{""message"":""Date cannot be set in the future"",""code"":0}",12,sever043,2019
6,22,2019-08-25 00:40:48,api_call_error,1,404,"{""message"":""Date cannot be set in the future"",""code"":0}",11,andreyvict,2019
7,26,2019-08-25 00:40:48,api_call_error,1,404,"{""message"":""Date cannot be set in the future"",""code"":0}",09,andreyvict,2019
8,31,2019-08-25 00:40:48,api_call_error,1,404,"{""message"":""Date cannot be set in the future"",""code"":0}",10,andreyvict,2019
9,33,2019-08-25 00:40:48,api_call_error,1,404,"{""message"":""Date cannot be set in the future"",""code"":0}",12,sever043,2019


In [75]:
ds=learn_prepair(df, 'Rosolimo')
#ds=learn_prepair(df, 'sever043')

In [76]:
ds.sample(3)

,Unnamed: 0,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url,target_user,score,rating,opponent_rating,base_elo_forec,color,year,month
57,0,chess,blitz,300,True,1,1-0,2016-09-16,19:53:50,2016-9,buzanin,Rosolimo,1608,1651,win,resigned,1.0,0.0,0.438432,0.561568,A00,https://www.chess.com/openings/A00-Grob-Opening-Grob-Gambit,https://www.chess.com/live/game/1732040676,https://api.chess.com/pub/player/buzanin,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.0,1651,1608,0.561568,b,2016,9
14,0,chess,blitz,180+2,True,1,1-0,2017-04-05,18:17:56,2017-4,zingarobarone,Rosolimo,1658,1534,win,resigned,1.0,0.0,0.671241,0.328759,A08,https://www.chess.com/openings/A08-Kings-Indian-Attack-French-Variation-4.d4,https://www.chess.com/live/game/2034406588,https://api.chess.com/pub/player/zingarobarone,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.0,1534,1658,0.328759,b,2017,4
209,0,chess,blitz,300,True,1,1-0,2013-10-24,12:41:27,2013-10,P4ULIU5,Rosolimo,1529,1486,win,resigned,1.0,0.0,0.561568,0.438432,D93,https://www.chess.com/openings/D93-Gruenfeld-Defense-Hungarian-Attack-5...O-O-6.e3,https://www.chess.com/live/game/629463175,https://api.chess.com/pub/player/p4uliu5,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.0,1486,1529,0.438432,b,2013,10


In [77]:
len(ds)

11504

In [78]:
ds.to_csv('data/cur_user.csv')

In [79]:
ds.groupby(['year', 'month']).sum()[['game', 'score']]#.plot()

game  score
year month             
2012 9        95   55.0
2013 9       215  105.5
     10      284  137.5
     11      287  146.0
     12      384  183.5
2014 1       355  182.5
     2       294  132.5
     3       262  127.0
     4       173   85.5
     5       202   89.5
     6       182   89.0
     7       222  112.0
     8       308  151.0
     9       149   70.5
     10       88   48.0
     11      193   88.0
     12      209  103.0
2015 1       147   72.0
     2       168   80.0
     3       189   96.0
     4       191   91.0
     5       222  106.0
     6        99   52.5
     7       349  172.5
     8       192   89.0
     9       258  119.0
     10      181   93.0
     11      185   87.0
     12      250  126.0
2016 1       247  117.0
     2       234  114.5
     3       109   49.0
     4       136   71.0
     5       130   62.5
     6        10    4.0
     7       113   58.0
     8       183   92.0
     9       114   50.5
     10      207  104.0
     11      116   52.0
     12      186   96.5
2017 1       158   82.0
     2        62   34.5
     3        94   41.5
     4       201  102.5
     5        56   24.5
     6        55   27.5
     7        84   45.5
     8        88   49.0
     9        77   44.5
     10       76   36.0
     11       16   11.0
     12       32   17.0
2018 1        54   29.0
     2        33   18.5
     3        80   42.0
     4        99   51.5
     5       189   99.0
     6       194  103.0
     7       124   57.5
     8       128   64.0
     9       177   90.0
     10      177   84.0
     11      164   85.0
     12      252  122.5
2019 1       143   67.0
     2       114   55.5
     3       203  107.5
     4        90   39.0
     5        40   19.5
     6        34   17.0
     7        45   20.5
     8        47   28.0

In [18]:
# tst=get_api_data_to_str('Rosolimo', '2015', '11')